##**Imports**

In [ ]:
import os
# import pandas as pd
import torch
# import sklearn
# from IPython.core.display import display, HTML
# import glob
# import numpy as np
from torch.utils.data import Dataset, DataLoader
# import re
# import torch.nn.functional as F

##**Installing Transformers**

The `transformers` library is a popular open-source library developed by Hugging Face. It provides pre-trained models and various tools to work with state-of-the-art natural language processing (NLP) models, including those based on transformer architectures.

When you install `transformers` using the command `!pip -q install transformers`, you're adding this library to your Python environment. This enables you to easily access pre-trained transformer models, such as BERT, GPT, and many others, without having to train them from scratch. This is incredibly useful for tasks like text classification, language modeling, translation, and more.

The library also offers helpful utilities for tokenization, model loading, and fine-tuning, making it a go-to choice for many researchers and practitioners in the field of NLP and AI.

In [ ]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.1 MB/s eta 0:00:00


`sentencepiece` is a library for tokenizing natural language text into subword units. It's often used in conjunction with transformer-based models like BERT. When you install `sentencepiece` using the command `!pip install sentencepiece`, you're adding support for this tokenization method to your environment.

In the context of transformer models, tokenization is a crucial step. It involves breaking down a sequence of text into smaller units, usually words or subwords, to be processed by the model. `sentencepiece` is one of the tokenizers that allows for a flexible and efficient way of handling this task.

By combining `transformers` and `sentencepiece`, you're setting up your environment to work seamlessly with transformer-based NLP models that use subword tokenization.

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
from apikey_h import apikey_h

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']= apikey_h

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. `from transformers import MarianMTModel, MarianTokenizer`: This line imports the `MarianMTModel` and `MarianTokenizer` classes from the `transformers` library. These classes are specific to MarianMT, which is a multilingual transformer model for machine translation.

2. `model_name = 'Helsinki-NLP/opus-mt-ml-en'`: Here, you're defining the model name or identifier. In this case, it's specifying the 'opus-mt-ml-en' model provided by Helsinki-NLP. This model is capable of translating between Malayalam and English.

3. `model = MarianMTModel.from_pretrained(model_name)`: This line initializes a `MarianMTModel` using the pre-trained weights specified by the `model_name`. It loads the pre-trained parameters of the model so that you can use it for translation without training it from scratch.

4. `tokenizer = MarianTokenizer.from_pretrained(model_name)`: Similarly, this line initializes a `MarianTokenizer` using the pre-trained tokenizer associated with the specified `model_name`. The tokenizer is essential for processing input text into a format that the model can understand.

Now you have a pre-trained MarianMT model and its corresponding tokenizer ready to use for machine translation tasks.


```python
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)
```

automatically downloads the pre-trained weights (model parameters) and associated tokenizer files for the specified `model_name` if they are not already present in your local environment. The downloaded files are typically stored in a cache directory so that they don't need to be downloaded again if you use the same model in the future.

Now that you've executed these lines, you have a pre-trained MarianMT model (`model`) and its corresponding tokenizer (`tokenizer`) ready to use for translation tasks.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
model_name = 'Helsinki-NLP/opus-mt-ml-en'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
import json

# File path to your JSONL file
file_path = '/content/drive/MyDrive/Colab Notebooks/Marian/dataset_everything.jsonl'

malayalam_sentences = []
english_sentences = []

with open(file_path, 'r') as file:
    for line in file:
        try:
            # Parse each line as a JSON object
            message = json.loads(line)

            # Extract Malayalam and English sentences
            if "messages" in message:
                for msg in message["messages"]:
                    if "role" in msg and "content" in msg:
                        if msg["role"] == "user":
                            malayalam_sentences.append(msg["content"])
                        elif msg["role"] == "assistant":
                            english_sentences.append(msg["content"])

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")


# Tokenize sentences
malayalam_tokenized = [tokenizer.encode(sentence, return_tensors="pt")[0] for sentence in malayalam_sentences]
english_tokenized = [tokenizer.encode(sentence, return_tensors="pt")[0] for sentence in english_sentences]


# Truncate sequences to a specified max length (e.g., 512)
max_length = 512
malayalam_tokenized = [sequence[:max_length] for sequence in malayalam_tokenized]
english_tokenized = [sequence[:max_length] for sequence in english_tokenized]

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, malayalam_tokenized, english_tokenized):
        self.malayalam_tokenized = malayalam_tokenized
        self.english_tokenized = english_tokenized

    def __len__(self):
        return len(self.malayalam_tokenized)

    def __getitem__(self, idx):
        return {
            "input_ids": self.malayalam_tokenized[idx],
            "labels": self.english_tokenized[idx],
        }

def collate_fn(batch):
    input_ids = pad_sequence([item["input_ids"] for item in batch], batch_first=True)
    labels = pad_sequence([item["labels"] for item in batch], batch_first=True)

    return {"input_ids": input_ids, "labels": labels}

# Create datasets and dataloaders
translation_dataset = TranslationDataset(malayalam_tokenized, english_tokenized)
batch_size = 8  # Adjust as needed
translation_loader = DataLoader(translation_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


In [ ]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup

# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-4 )
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(translation_loader))

# Training loop
num_epochs = 3  # Adjust as needed

for epoch in range(num_epochs):
    print ('epoch:', epoch)
    model.train()


    for batch in translation_loader:
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        print (loss)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/OCR_Sale_Deed')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch: 0
tensor(3.3999, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7605, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5130, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5635, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.8422, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9377, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.9266, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.1473, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.1606, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8746, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4518, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.2629, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.2898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6494, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.0572, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.8450, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.3754, device='cuda:0',

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Marian/model_finetuned')

In [ ]:
# Malayalam sentence
malayalam_sentence = '''ഇന്ന് നല്ല കാലാവസ്ഥ ആണ്  '''

tokenized_input = tokenizer.encode(malayalam_sentence, return_tensors="pt")

tokenized_input = tokenized_input.to(device)

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Generate translations
    generated = model.generate(tokenized_input)

generated_translations = tokenizer.decode(generated[0], skip_special_tokens=True)
print("Generated Translations:", generated_translations)

Generated Translations: 
